In [1]:
# coding: utf-8
"""
Beating the benchmark @ Kaggle Springleaf
Basis code provided by Abhishek Thakur
Further edits made by Wendell Kuling
"""

'\nBeating the benchmark @ Kaggle Springleaf\nBasis code provided by Abhishek Thakur\nFurther edits made by Wendell Kuling\n'

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import re
from datetime import datetime

from sklearn import preprocessing, linear_model
%matplotlib inline

In [86]:
train = pd.read_csv('~/ipython/Kaggle_springleaf/Input/train.csv')
test = pd.read_csv('~/ipython/Kaggle_springleaf/Input/test.csv')

In [87]:
y = train.target.values
train = train.drop(['ID', 'target'], axis=1)
test = test.drop('ID', axis=1)

In [88]:
train = train.dropna(axis=1, thresh=2000)
test = test.dropna(axis=1, thresh=2000)

train = train.fillna(-1)
test = test.fillna(-1)

In [89]:
# Helper function to process some of the day observations
def timetodaysago(timeobs):
    try:
        d0 = datetime.strptime(timeobs[:7], '%d%b%y')
        d1 = datetime.strptime('01SEP15', '%d%b%y')
        delta = d1 - d0
        return delta.days
    except:
        return timeobs

In [90]:
def isdate(seriestocheck):
    p = re.compile(r'\w{7}:\d{2}:\d{2}:\d{2}') # regex for our datetime format
    for check in seriestocheck:
        if p.match(str(check)):
            return True

In [91]:
for f in train.columns:
    top_values = pd.Series(list(train[f].value_counts().keys()))[:10]
    if isdate(top_values):
        print "Variable: " + f + " is of type date. Converting."
        train[f]=train[f].map(timetodaysago)
        train[f]=train[f].astype('float64')
        test[f]=test[f].map(timetodaysago)
        test[f]=test[f].astype('float64')

Variable: VAR_0073 is of type date. Converting.
Variable: VAR_0075 is of type date. Converting.
Variable: VAR_0156 is of type date. Converting.
Variable: VAR_0158 is of type date. Converting.
Variable: VAR_0159 is of type date. Converting.
Variable: VAR_0166 is of type date. Converting.
Variable: VAR_0167 is of type date. Converting.
Variable: VAR_0168 is of type date. Converting.
Variable: VAR_0169 is of type date. Converting.
Variable: VAR_0176 is of type date. Converting.
Variable: VAR_0177 is of type date. Converting.
Variable: VAR_0178 is of type date. Converting.
Variable: VAR_0179 is of type date. Converting.
Variable: VAR_0204 is of type date. Converting.
Variable: VAR_0217 is of type date. Converting.


In [92]:
# Add a row wise count of NA’s 

train['NumbNAs'] = train.apply(lambda x : x[x==-1].count(), axis = 1)
test['NumbNAs'] = train.apply(lambda x : x[x==-1].count(), axis = 1)

In [93]:
# and row wise count of outliers (those 999…’s). 

train['Numb99s'] = train.apply(lambda x : x.astype(str).str.contains('99').sum(), axis = 1)
test['Numb99s'] = test.apply(lambda x : x.astype(str).str.contains('99').sum(), axis = 1)

In [110]:
# Zip code var 241, 212 (+ID?), 274, 237
# Dropping the Zip codes and keeping the values in 274, 237

temp = train[['VAR_0274', 'VAR_0237']]
temp['target'] = y

temp = temp.applymap(lambda x: np.nan if x == -1 else x)

tempmean_237 = temp.groupby(['VAR_0237']).mean()
searchmean_237 = tempmean_237['target'].to_dict()

tempmean_274 = temp.groupby(['VAR_0274']).mean()
searchmean_274 = tempmean_274['target'].to_dict()

train['at_237'] = train['VAR_0237'].map(lambda x: searchmean_237[x] if (x in searchmean_237) else -1)
test['at_237'] = test['VAR_0237'].map(lambda x: searchmean_237[x] if (x in searchmean_237) else -1)

train['at_274'] = train['VAR_0274'].map(lambda x: searchmean_274[x] if (x in searchmean_274) else -1)
test['at_274'] = test['VAR_0274'].map(lambda x: searchmean_274[x] if (x in searchmean_274) else -1)


float64


/Users/wendellkuling/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [111]:
train.drop(['VAR_0241', 'VAR_0212'], axis = 1, inplace = True)
test.drop(['VAR_0241', 'VAR_0212'], axis = 1, inplace = True)

In [112]:
# From https://www.kaggle.com/darraghdog/springleaf-marketing-response/grouping-numerics-springleaf
# We know that VAR_0254, VAR_0255 and VAR_0198 are age features
# Creating three columns: one with the best estimate for age, one with avg target score per age-category, 
# one with avg stdev of target per age-category 
# dropped the stdev one after experimentation

temp = train[['VAR_0254', 'VAR_0255', 'VAR_0198']]
temp = temp.applymap(lambda x: np.nan if x == -1 else x)
temp['age'] = temp.min(axis='columns')
temp['age'] = temp['age'].astype(np.float64)
temp.drop(['VAR_0254', 'VAR_0255', 'VAR_0198'], axis = 1, inplace = True)

temp['target'] = y

tempmean = temp.groupby(['age']).mean()
searchmean = tempmean['target'].to_dict()

# tempstd = temp.groupby(['age']).std()
# searchstd = tempstd['target'].to_dict()

temp['at_age'] = temp['age'].map(lambda x: searchmean[x] if (not pd.isnull(x)) else -1)
# temp['st_age'] = temp['age'].map(lambda x: searchstd[x] if (not pd.isnull(x)) else -1)

train['age'] = temp['age']
train['at_age'] = temp['at_age']
# train['st_age'] = temp['st_age']


temp = test[['VAR_0254', 'VAR_0255', 'VAR_0198']]
temp = temp.applymap(lambda x: np.nan if x == -1 else x)
temp['age'] = temp.min(axis='columns')
temp['age'] = temp['age'].astype(np.float64)
temp['at_age'] = temp['age'].map(lambda x: searchmean[x] if (x in searchmean) else -1)
# temp['st_age'] = temp['age'].map(lambda x: searchstd[x] if (x in searchstd) else -1)

test['age'] = temp['age']
test['at_age'] = temp['at_age']
# test['st_age'] = temp['st_age']

In [113]:
train.drop(['VAR_0254', 'VAR_0255', 'VAR_0198'], axis = 1, inplace = True)
test.drop(['VAR_0254', 'VAR_0255', 'VAR_0198'], axis = 1, inplace = True)

In [114]:
for f in train.columns:
    if train[f].dtype=='object': 
        print f
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))                    

VAR_0001
VAR_0005
VAR_0008
VAR_0009
VAR_0010
VAR_0011
VAR_0012
VAR_0043
VAR_0044
VAR_0196
VAR_0200
VAR_0202
VAR_0216
VAR_0222
VAR_0226
VAR_0229
VAR_0230
VAR_0232
VAR_0236
VAR_0237
VAR_0239
VAR_0274
VAR_0283
VAR_0305
VAR_0325
VAR_0342
VAR_0352
VAR_0353
VAR_0354
VAR_0404
VAR_0466
VAR_0467
VAR_0493
VAR_1934


In [117]:
X = np.array(train)
X_test = np.array(test)
clf = xgb.XGBClassifier(n_estimators=5000, nthread=-1, max_depth=17, 
                        learning_rate=0.01, silent=False, subsample=0.8, colsample_bytree=0.7)

In [118]:
clf.fit(X, y, eval_metric = 'auc', verbose = True)

XGBClassifier(base_score=0.5, colsample_bytree=0.7, gamma=0,
       learning_rate=0.01, max_delta_step=0, max_depth=17,
       min_child_weight=1, missing=None, n_estimators=5000, nthread=-1,
       objective='binary:logistic', seed=0, silent=False, subsample=0.8)

In [119]:
preds = clf.predict_proba(X_test)[:,1]
sample = pd.read_csv('~/ipython/Kaggle_springleaf/Input/sample_submission.csv')
sample.target = preds
sample.to_csv('20151031v3_benchmarkxgb5000.csv', index=False)

In [26]:
import pickle

with open('5000model', 'wb') as handle:
    pickle.dump(clf, handle)

In [ ]:
# To do's:

# For binary features > T-SNE clustering on it

# (2) from step 1 i got a feature importance list, then split all the ranked feature set into 5 groups by idx mod 5.
# Each file would lead a result of ~0.798 in LB, but ensemble these 5 files would lead to 0.8255 in LB.
# This is very interesting.


# Set evaluation metric to AUC

# params = {"objective": "binary:logistic", "eta": 0.015, "max_depth": 22, "min_child_weight": 3, 
#      "silent": 1, "subsample": 0.7, "colsample_bytree": 0.7, "seed": 231, "eval_metric":"auc"} num_rounds = 2125

# list(objective = "binary:logistic", eta = 0.0025, max_depth = 15, 
# subsample = 0.7, colsample_bytree = 0.5, min_child_weight = 4, eval_metric = "auc", alpha = 1)


# Build ensemble



In [ ]:
def get_ctr_features(data, test, y, ctr_cols, dctr, num):
        data["target"] = y
        dcols = set(test.columns)
        kf = cross_validation.StratifiedKFold(y, n_folds=4, shuffle=True, random_state=11)
        tr = np.zeros((data.shape[0], len(ctr_cols)))
        for kfold, (itr, icv) in enumerate(kf):
            data_tr = data.iloc[itr]
            data_te = data.iloc[icv]
            for t, col in enumerate(ctr_cols):
                if col not in dcols:
                    continue
                ctr_df = data_tr[[col, "target"]].groupby(col).agg(["count", "sum"])
                ctr_dict = ctr_df.apply(lambda x: calc_ctr(x, num), axis=1).to_dict()
                tr[icv, t] = data_te[col].apply(lambda x: ctr_dict.get(x, dctr))

        te = np.zeros((test.shape[0], len(ctr_cols)))
        for t, col in enumerate(ctr_cols):
            if col not in dcols:
                    continue
            ctr_df = data[[col, "target"]].groupby(col).agg(["count", "sum"])
            ctr_dict = ctr_df.apply(lambda x: calc_ctr(x, num), axis=1).to_dict()
            te[:, t] = test[col].apply(lambda x: ctr_dict.get(x, dctr))
        del data["target"]
        return tr, te